In [2]:
import json
import random

from collections import Counter

from allennlp.predictors.predictor import Predictor

# from utils import sent_to_conll10

In [3]:
model_url = "https://storage.googleapis.com/allennlp-public-models/coref-spanbert-large-2021.03.10.tar.gz"
predictor = Predictor.from_path(model_url)

2021-06-28 09:32:53 INFO: Downloading default packages for language: en (English)...
2021-06-28 09:32:54 INFO: File exists: /home/he/stanza_resources/en/default.zip.
2021-06-28 09:32:57 INFO: Finished downloading models and saved to /home/he/stanza_resources.
2021-06-28 09:32:57 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| pos       | combined  |
| lemma     | combined  |
| depparse  | combined  |
| sentiment | sstplus   |
| ner       | ontonotes |

2021-06-28 09:33:00 INFO: Use device: cpu
2021-06-28 09:33:00 INFO: Loading: tokenize
2021-06-28 09:33:00 INFO: Loading: pos
2021-06-28 09:33:00 INFO: Loading: lemma
2021-06-28 09:33:00 INFO: Loading: depparse
2021-06-28 09:33:00 INFO: Loading: sentiment
2021-06-28 09:33:00 INFO: Loading: ner
2021-06-28 09:33:01 INFO: Done loading processors!
Some weights of BertModel were not initialized from the model checkpoint at SpanBERT/spanbert-large-cased and a

In [4]:
with open("./data/subtask3-coreference/en-train.json") as fp:
    entries = fp.readlines()

In [5]:
data = []
for entry in entries:
    data.append(json.loads(entry))

In [6]:
print("Examples in the dataset:")
random.sample(data, 2)

Examples in the dataset:


[{'event_clusters': [[12, 13], [3, 14]],
  'sentence_no': [3, 12, 13, 14],
  'sentences': ['The PLGA week coincides with the 48 - hour bandh call by the Maoists protesting the killing of Kishenji .',
   'Though rebels frequent the villages including Karada , Ranaba and Indragada , this is for the first time that posters exhorting the villagers to participate in the PLGA week have been found at several places in the block .',
   'The posters have warned them against joining the police and army and threatened to take up such cases in the praja court .',
   'With PLGA week coinciding with the two-day bandh call from December 4 , protesting the killing of Kishenji , the villagers apprehend'],
  'id': 55456},
 {'event_clusters': [[1, 13, 14, 19, 23, 25]],
  'sentence_no': [1, 13, 14, 19, 23, 25],
  'sentences': ["Two men who confessed to being involved in the murder of ANC North West official Obuti Chika were tortured by police , the Klerksdorp Magistrate 's Court heard on Tuesday .",
   'P

In [7]:
print("Number of entries")
len(data)

Number of entries


596

In [8]:
sentence_no_list = []
event_clusters_list = []

for d in data:
    sentence_no_list += d['sentence_no']
    event_clusters_list += d['event_clusters']

In [9]:
# genre_list = list(set(sentence_no_list))
# model_name = "spanbert_base"

In [10]:
prediction = predictor.predict(document='\t'.join(data[0]['sentences']))

In [11]:
def my_prediction(data_entry):
    sentences = data_entry['sentences']
    sentences_no_mark = []
    for no, sent in enumerate(sentences):
        # _len = len(sent.split(' '))
        _id = data_entry['sentence_no'][no]
        # _mark = [_id]*_len
        # _mark.append('<SEP>')
        # sentences_no_mark += _mark

    _pred = predictor.predict(document='\n'.join(sentences))
    _clusters = _pred['clusters']
    _docu = _pred['document']
    _idx = 0
    for _w in _docu:
        if _w != '\n':
            sentences_no_mark.append(data_entry['sentence_no'][_idx])
        else:
            _idx += 1
            sentences_no_mark.append('<SEP>')

    _sent_clusters = []
    _sent_clusters_words = []
    for cs in _clusters:
        _sent_cluster = []
        _sent_cluster_words = []
        for c in cs:
            _sent_cluster.append(sentences_no_mark[c[0]])
            if c[0] == c[1]:
                _sent_cluster_words.append(_docu[c[0]])
            else:
                _sent_cluster_words.append(' '.join(_docu[c[0]:c[1]+1]))
        # _sent_cluster = sorted(list(set(_sent_cluster)))
        _sent_clusters.append(_sent_cluster)
        _sent_clusters_words.append(_sent_cluster_words)
    return _sent_clusters, _sent_clusters_words




In [12]:
test_entry = data[23]
my_prediction(test_entry)


([[1, 1, 2, 2, 6],
  [1, 1, 2, 3, 4, 5],
  [1, 1, 4],
  [2, 3, 4, 5],
  [1, 3],
  [2, 5]],
 [['NSCN(K )',
   'Naga insurgent outfit NSCN ( Khaplang ) , with which the Centre abrogated ceasefire in March',
   'The militant outfit',
   'it',
   'NSCN(K )'],
  ['Ambush on Army in Manipur',
   'the ambush in Manipur in which 18 army personnel were killed and 10 others injured',
   'the attack yesterday',
   'the incident',
   'the attack',
   'The assault'],
  ['Manipur', 'Manipur', 'Manipur'],
  ['a release dated June 4 , which was circulated in the media today',
   'The release',
   'It',
   'the release'],
  ['army', 'army'],
  ['it , Kanglei Yawol Kanna Lup and Kangleipak Communist Party',
   'the outfits']])

1	NSCN	NSCN	PROPN	NNP	Number=Sing	5	nsubj	_	start_char=0|end_char=4
2	(	(	PUNCT	-LRB-	_	3	punct	_	start_char=4|end_char=5
3	K	k	NUM	LS	_	5	nummod	_	start_char=5|end_char=6
4	)	)	PUNCT	-RRB-	_	3	punct	_	start_char=6|end_char=7
5	Claims	claim	VERB	VBZ	Mood=Ind|Number=Sing|Person=3|Tense=Pres|VerbForm=Fin	0	root	_	start_char=8|end_char=14
6	Responsibility	responsibility	NOUN	NN	Number=Sing	5	obj	_	start_char=15|end_char=29
7	for	for	ADP	IN	_	8	case	_	start_char=30|end_char=33
8	Ambush	Ambush	PROPN	NNP	Number=Sing	6	nmod	_	start_char=34|end_char=40
9	on	on	ADP	IN	_	10	case	_	start_char=41|end_char=43
10	Army	Army	PROPN	NNP	Number=Sing	8	nmod	_	start_char=44|end_char=48
11	in	in	ADP	IN	_	12	case	_	start_char=49|end_char=51
12	Manipur	Manipur	PROPN	NNP	Number=Sing	6	nmod	_	start_char=52|end_char=59
13	05th	05th	NUM	CD	NumType=Card	14	nummod	_	start_char=60|end_char=64
14	June	June	PROPN	NNP	Number=Sing	12	compound	_	start_char=65|end_char=69
15	2015	2015	NUM	CD	NumType=Card	14	nummod	_	start